In [1]:

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import plotly.express as px
from datetime import datetime
from io import StringIO
import base64


In [2]:

# Login UI
username_input = widgets.Text(placeholder='Username', description='Username:')
password_input = widgets.Password(placeholder='Password', description='Password:')
login_button = widgets.Button(description='Login', button_style='success')
login_output = widgets.Output()

login_box = widgets.VBox([username_input, password_input, login_button, login_output])
login_box.layout.align_items = 'center'
# Wrap login_box in a vertically and horizontally centered container
login_container = widgets.VBox([login_box])
login_container.layout = widgets.Layout(
    height='100vh',
    justify_content='center',
    align_items='center',
    display='flex',
    flex_flow='column'
)


In [3]:

# Dashboard UI (from dashboard.ipynb)
def show_dashboard():
    # ----- CSS Styling -----
    style_html = """
    <style>
        .main-wrap {
            display: flex;
            justify-content: center;
            margin-top: 40px;
        }
        .centered-box {
            background-color: #f0f8ff;
            padding: 30px;
            border-radius: 12px;
            width: 600px;
            box-shadow: 0 4px 10px rgba(0,0,0,0.1);
            font-family: Arial, sans-serif;
        }
        h2 {
            text-align: center;
            margin-bottom: 20px;
            color: #333;
        }
        .output-centered {
            display: flex;
            justify-content: center;
            margin-top: 20px;
        }
    </style>
    <div class="main-wrap">
      <div class="centered-box">
        <h2>💸 BudgetBuddy+ Expense Tracker</h2>
      </div>
    </div>
    """
    display(HTML(style_html))
desc_input = widgets.Text(description="Desc", layout=widgets.Layout(width='200px'))
amount_input = widgets.FloatText(description="Amount", layout=widgets.Layout(width='150px'))
category_input = widgets.Dropdown(
    options=['Food', 'Transport', 'Utilities', 'Shopping', 'Other'],
    description='Category',
    layout=widgets.Layout(width='200px')
)
date_input = widgets.DatePicker(
    description='Date',
    value=datetime.today(),
    layout=widgets.Layout(width='200px')
)
budget_input = widgets.FloatText(description="Budget $", layout=widgets.Layout(width='200px'), value=500)

add_button = widgets.Button(description="Add Expense", button_style='success', layout=widgets.Layout(width='150px'))
reset_button = widgets.Button(description="Reset", button_style='danger', layout=widgets.Layout(width='150px'))
upload_button = widgets.FileUpload(accept='.csv', multiple=False)
download_button = widgets.Button(description="Download CSV", button_style='info', layout=widgets.Layout(width='200px'))
logout_button = widgets.Button(description="Logout", button_style='danger', layout=widgets.Layout(width='150px'))

output = widgets.Output(layout=widgets.Layout(width='600px'))
expenses = []

def add_expense(b):
    if desc_input.value and amount_input.value:
        expenses.append({
            'Date': date_input.value.strftime('%Y-%m-%d') if date_input.value else '',
            'Description': desc_input.value,
            'Amount': amount_input.value,
            'Category': category_input.value
        })
        update_display()

def reset_expenses(b):
    expenses.clear()
    update_display()

def upload_csv(change):
    if upload_button.value:
        file_content = list(upload_button.value.values())[0]['content']
        df = pd.read_csv(StringIO(file_content.decode('utf-8')))
        for _, row in df.iterrows():
            expenses.append({
                'Date': row.get('Date', ''),
                'Description': row.get('Description', ''),
                'Amount': float(row.get('Amount', 0)),
                'Category': row.get('Category', 'Other')
            })
        update_display()

def download_csv(b):
    if expenses:
        df = pd.DataFrame(expenses)
        csv_data = df.to_csv(index=False)
        b64 = base64.b64encode(csv_data.encode()).decode()
        href = f'<a download="expenses.csv" href="data:text/csv;base64,{b64}" target="_blank">📥 Click here to download CSV</a>'
        with output:
            display(HTML(href))

def update_display():
    with output:
        clear_output()
        if expenses:
            df = pd.DataFrame(expenses)
            display(df)
            total = df['Amount'].sum()
            budget = budget_input.value
            print(f"Total: ${total:.2f}")
            if total > budget:
                print(f"⚠️ You are over budget! (${budget:.2f})")
            else:
                print(f"✅ Within budget (${budget:.2f})")
            fig = px.pie(df, names='Category', values='Amount', title='Expenses by Category')
            fig.show()
        else:
            print("No expenses yet.")

add_button.on_click(add_expense)
reset_button.on_click(reset_expenses)
upload_button.observe(upload_csv, names='value')
download_button.on_click(download_csv)

dashboard_ui = widgets.VBox([
    widgets.HTML("<h2>📊 Welcome to BudgetBuddy+ Dashboard</h2>"),
    widgets.HBox([desc_input, amount_input]),
    widgets.HBox([category_input, date_input]),
    widgets.HBox([budget_input]),
    widgets.HBox([add_button, reset_button]),
    widgets.HBox([upload_button, download_button]),
    output,
    logout_button
])
dashboard_ui.layout.align_items = 'center'
dashboard_container = widgets.VBox([dashboard_ui])
dashboard_container.layout = widgets.Layout(
    height='100vh',
    justify_content='center',
    align_items='center',
    display='flex',
    flex_flow='column'
)

dashboard_container.layout.align_items = 'center'
dashboard_container.layout.display = 'none'





In [4]:

import psycopg2

def authenticate_user(username, password):
    try:
        # Update these values with your actual PostgreSQL credentials
        connection = psycopg2.connect(
            host="localhost",
            database="myapp",
            user="postgres",
            password="1234"
        )
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM users WHERE username=%s AND password=%s", (username, password))
        user = cursor.fetchone()
        cursor.close()
        connection.close()
        return user is not None

    except Exception as e:
        with login_output:
            clear_output()
            print("❌ Database connection error:", str(e))
        return False


In [5]:

def handle_login(b):
    with login_output:
        clear_output()
        if authenticate_user(username_input.value, password_input.value):
            login_box.layout.display = 'none'
            dashboard_container.layout.display = 'flex'
            display(form_box)
        else:
            print("❌ Incorrect username or password!")

def handle_logout(b):
    dashboard_container.layout.display = 'none'
    login_box.layout.display = 'flex'
    username_input.value = ''
    password_input.value = ''
    with login_output:
        clear_output()

login_button.on_click(handle_login)
logout_button.on_click(handle_logout)


In [6]:


# Combine login and dashboard in a master container
app = widgets.VBox([login_container, dashboard_container])
app.layout.align_items = 'center'
display(app)
